### prueba scrapear!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import os

In [2]:
ua = UserAgent()
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

#### carrefour

In [ ]:
list_pages_carrefour = list(range(0,336,24))
table_pages_carrefour=[]
for number in list_pages_carrefour:
    url_carrefour = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url_carrefour, headers=header).text
    soup_carrefour = BeautifulSoup(html, 'lxml')
    table_carrefour = soup_carrefour.find_all({'article':'product-card-item'})
    table_pages_carrefour.append(table_carrefour)

In [ ]:
table_pages_carrefour[0][23]

In [ ]:
len(table_pages_carrefour[0])

In [ ]:
data_raw_carrefour=[]
for i in range(len(table_pages_carrefour)):
    for j in range(len(table_pages_carrefour[i])):
        rows_carrefour = table_pages_carrefour[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion_carrefour = table_pages_carrefour[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows_carrefour) != 0:
            if len(promotion_carrefour) != 0:
                data_raw_carrefour.append(rows_carrefour + promotion_carrefour)
            else:
                data_raw_carrefour.append(rows_carrefour)
                
            

In [ ]:
data_raw_carrefour[20]

In [ ]:
len(data_raw_carrefour)

In [ ]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return float(re.sub(",",'.',price))

In [ ]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return float(price_liter)

In [ ]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [ ]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [ ]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6]

In [ ]:
#container (carrefour  / alcampo)
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [ ]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [ ]:
#quantity (carrefour  / alcampo)
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de|pack',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [ ]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [ ]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw_carrefour)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_carrefour)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw_carrefour[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw_carrefour[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw_carrefour[beer_number][0])
    data_beer_carrefour.iloc[beer_number,9] = 'Carrefour'
    if len(data_raw_carrefour[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw_carrefour[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [ ]:
data_beer_carrefour.head()

In [ ]:
sum(data_beer_carrefour['price'])

## Alcampo

In [ ]:
list_pages_alcampo = list(range(0,10))
table_pages_alcampo=[]
for number in list_pages_alcampo:
    url_alcampo = f'https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page={number}'
    html = requests.get(url_alcampo, headers=header).text
    soup_alcampo = BeautifulSoup(html, 'lxml')
    table_alcampo = soup_alcampo.find_all('div',{'class':'productGridItem'})
    table_pages_alcampo.append(table_alcampo)
    

In [ ]:
len(table_pages_alcampo[0])

In [ ]:
len(table_pages_alcampo)

In [ ]:
data_raw_alcampo=[]
for i in range(len(table_pages_alcampo)):
    for j in range(len(table_pages_alcampo[i])):
        rows_alcampo = table_pages_alcampo[i][j].find_all(['span','div'],{'class': ['price',
                                                                                    'productName',
                                                                                    'thumb cut-alt-img']})
        promotion_alcampo = table_pages_alcampo[i][j].find_all('div', {'class': 'financiacionMensual'})
        if len(rows_alcampo) != 0:
            if len(promotion_alcampo) != 0:
                data_raw_alcampo.append(rows_alcampo + promotion_alcampo)
            else:
                data_raw_alcampo.append(rows_alcampo)
                

In [ ]:
len(data_raw_alcampo)

In [ ]:
#price alcampo / corte ingles / Dia
def get_price_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    return float(re.sub(",",'.',price[0]))


In [ ]:
#price_liter alcampo / corte ingles / Dia
def get_price_liter_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    if len(price)==1:
        return float(re.sub(",",'.',price[0]))
    else:
        return float(re.sub(",",'.',price[1]))


In [ ]:
#title alcampo / corte ingles
def get_tittle_alcampo(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\n','',tittle_raw)
    return tittle


In [ ]:
#promotion alcampo
def get_promotion_alcampo(data_text):
    promotion_raw = data_text.text
    promotion = re.sub('\r\n\r\n\t\t\t\t\t\t\t\t\t\t','',promotion_raw)
    return promotion

In [ ]:
#brand alcampo
def get_brand_alcampo(data_text):
    brand_raw = data_text.text
    brand = re.sub('\n','',brand_raw)
    brand_2 = re.findall('[A-Z 0-9,À-ÿ]{3,} ',brand)
    return brand_2[0]

In [ ]:
#volumen alcampo
def get_volumen_unid_alcampo(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9.,]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',volumen_raw)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [ ]:
#image alcampo
def get_image_alcampo(data_text):
    return data_text.find_all('img')[0].get('data-blzsrc')

In [ ]:
data_raw_alcampo[0][1]

In [ ]:
data_beer_alcampo= pd.DataFrame(index=range(0,len(data_raw_alcampo)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_alcampo)):
    data_beer_alcampo.iloc[beer_number,0] = get_price_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,2] = get_tittle_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,4] = get_brand_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,5] = get_container(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,7] = get_quantity_pack(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,8] = get_image_alcampo(data_raw_alcampo[beer_number][0])
    data_beer_alcampo.iloc[beer_number,9] = 'Alcampo'
    if len(data_raw_alcampo[beer_number]) > 3:
        data_beer_alcampo.iloc[beer_number,3] = get_promotion_alcampo(data_raw_alcampo[beer_number][3])
    else:
        data_beer_alcampo.iloc[beer_number,3] = 'No promotion'

In [ ]:
data_beer_alcampo.head(50)

## El corte ingles

In [ ]:
list_pages_corteingles = list(range(1,26))
table_pages_corteingles =[]
for number in list_pages_corteingles:
    url_corteingles = f'https://elcorteingles.es/supermercado/bebidas/cervezas/{number}/'
    html = requests.get(url_corteingles, headers=header).text
    soup_corteingles= BeautifulSoup(html, 'lxml')
    table_corteingles = soup_corteingles.find_all('div',{'class':'grid-item'})
    table_pages_corteingles.append(table_corteingles)

In [ ]:
len(table_pages_corteingles)

In [ ]:
len(table_pages_corteingles[25])

In [ ]:
data_raw_corteingles=[]
for i in range(len(table_pages_corteingles)):
    for j in range(len(table_pages_corteingles[i])):
        rows_corteingles = table_pages_corteingles[i][j].find_all(['div','h3'],{'class': ['prices-price',
                                                                                    'product_tile-description',
                                                                                    'product_tile-image _fade']})
        promotion_corteingles = table_pages_corteingles[i][j].find_all('div', {'class': 'product_tile-offer offer'})
        if len(rows_corteingles) > 3:
            if len(promotion_corteingles) != 0:
                data_raw_corteingles.append(rows_corteingles + promotion_corteingles)
            else:
                data_raw_corteingles.append(rows_corteingles)
                

In [ ]:
len(data_raw_corteingles)

In [ ]:
#brand corte ingles
def get_brand_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('cervez',brand_raw)
    return brand[0]

In [ ]:
#volumen corte ingles
def get_volumen_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('botel',brand_raw)
    return brand[0]

In [ ]:
#image corte ingles
def get_image_corteingles(data_text):
    return 'https:'+data_text.find_all('img')[0].get('src')

In [ ]:
data_raw_corteingles[2][3].text

In [ ]:
data_beer_corteingles = pd.DataFrame(index=range(0,len(data_raw_corteingles)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_corteingles)):
    data_beer_corteingles.iloc[beer_number,0] = get_price_alcampo(data_raw_corteingles[beer_number][1])
    data_beer_corteingles.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_corteingles[beer_number][2])
    data_beer_corteingles.iloc[beer_number,2] = get_tittle_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,4] = get_brand_corteingles(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,5] = get_container(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,7] = get_quantity_pack(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,8] = get_image_corteingles(data_raw_corteingles[beer_number][0])
    data_beer_corteingles.iloc[beer_number,9] = 'El Corte Ingles'
    if len(data_raw_corteingles[beer_number]) > 4:
        data_beer_corteingles.iloc[beer_number,3] = get_promotion_alcampo(data_raw_corteingles[beer_number][4])
        if len(data_raw_corteingles[beer_number]) > 5:
            data_beer_corteingles.iloc[beer_number,3] += '..SECOND OFFERT: '+ get_promotion_alcampo(data_raw_corteingles[beer_number][5])
    else:
        data_beer_corteingles.iloc[beer_number,3] = 'No promotion'    

In [ ]:
data_beer_corteingles.head()

## DIA

In [ ]:
list_pages_dia= list(range(0,3))
table_pages_dia =[]
for number in list_pages_dia:
    url_dia = f'https://www.dia.es/compra-online/productos/bebidas/cervezas/c/WEB.008.064.00000?page={number}&disp='
    html = requests.get(url_dia, headers=header).text
    soup_dia= BeautifulSoup(html, 'lxml')
    table_dia = soup_dia.find_all('div',{'class':'prod_grid'})
    table_pages_dia.append(table_dia)

In [ ]:
len(table_pages_dia)

In [ ]:
len(table_pages_dia[2])

In [ ]:
data_raw_dia=[]
for i in range(len(table_pages_dia)):
    for j in range(len(table_pages_dia[i])):
        rows_dia = table_pages_dia[i][j].find_all(['div','span'],{'class': ['price_container','details',
                                                                            'thumb']})
        if len(rows_dia) > 2:
            data_raw_dia.append(rows_dia)
                

In [ ]:
len(data_raw_dia)

In [ ]:
#title Dia
def get_tittle_dia(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\r\n\t\t\t\t','',tittle_raw)
    return tittle

In [ ]:
#brand Dia
def get_brand_dia(data_text):
    brand_raw = data_text.text
    brand_sub = re.sub('\r\n\t\t\t\t','',brand_raw)
    brand = re.split('cervez',brand_sub)
    return brand[0]

In [ ]:
#image dia
def get_image_dia(data_text):
    return data_text.find_all('img')[0].get('data-original')

In [ ]:
data_raw_dia[24][2].text

In [ ]:
data_beer_dia= pd.DataFrame(index=range(0,len(data_raw_dia)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_dia)):
    data_beer_dia.iloc[beer_number,0] = get_price_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,2] = get_tittle_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,4] = get_brand_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,5] = get_container(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,7] = get_quantity_pack(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,8] = get_image_dia(data_raw_dia[beer_number][0])
    data_beer_dia.iloc[beer_number,9] = 'Dia'
    if len(data_raw_dia[beer_number]) > 3:
        data_beer_dia.iloc[beer_number,3] = get_promotion_alcampo(data_raw_dia[beer_number][3])
    else:
        data_beer_dia.iloc[beer_number,3] = 'No promotion' 

In [ ]:
data_beer_dia.head()

## Eroski

In [49]:
os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}'  
os.environ['PATH']

driver_options = Options()
driver_options.headless = True
driver = webdriver.Firefox(options=driver_options)

driver.get('https://supermercado.eroski.es/es/supermercado/2060211-bebidas/2060233-cervezas/') 
for i in range(20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    
selenium_raw_eroski = driver.find_elements_by_class_name('product-item')

In [50]:
len(selenium_raw_eroski)

276

In [52]:
data_raw_eroski = []
for i in selenium_raw_eroski:
    data_raw_eroski.append(i.text)

In [56]:
#price eroski

def get_price_eroski(data_text):
    price = re.findall('[0-9,]+',data_text)
    last = len(price)-1
    return float(re.sub(",",'.',price[last]))

In [76]:
#price_liter eroski

def get_price_liter_eroski(data_text):
    price_raw = re.split(' LITRO A ',data_text)
    if len(price_raw)>1:
        price = re.findall('[0-9,]+',price_raw[1])
    else:
        price = re.findall('[0-9,]{2,}',data_text)
    return float(re.sub(",",'.',price[0]))

In [82]:
#title eroski
def get_tittle_eroski(data_text):
    tittle = re.split('\n',data_text)
    return tittle[1]

In [167]:
#brand eroski
def get_brand_eroski(data_text):
    brand_raw = re.split('rveza ',data_text)
    if len(brand_raw)>1:
        brand = re.findall('[A-Z 0-9À-ÿ]{3,}',brand_raw[1])
        return brand[0]
    else:
        brand = re.findall('[A-Z 0-9À-ÿ]{3,}',data_text)
        return brand[1]


In [174]:
#container eroski
def get_container_eroski(data_text):
    container = re.findall('botella|lata|barril',data_text)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [178]:
#volumen eroski
def get_volumen_unid_eroski(data_text):
    volumen = re.findall('[0-9.,]+ cl',data_text)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',data_text)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [183]:
#quantity eroski
def get_quantity_pack_eroski(data_text):
    quantity_split = re.split('pack de|pack',data_text)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [221]:
#promotion eroski

def get_promotion_eroski(data_text):
        promotion = re.split("\n",data_text)
        if len(promotion)>8:
            number = len(promotion)-6
            return promotion[number] + ' ' + promotion[number+1]
        else:
            return 'No promotion'


In [222]:
data_raw_eroski[4]

'  0\nCerveza SAN MIGUEL, pack botellín 12x25 cl\n1 LITRO A 1,76 €\n2ª unidad\n-50%\nAhora\n5,31 €\nAñadir\nAñadir a la lista'

In [226]:
data_raw_eroski[3]

'  0\nCerveza MAHOU 5 Estrellas, pack 24x33 cl\n1 LITRO A 0,97 €\n13,59 €\nAntes\nAhora\n7,75 €\nAñadir\nAñadir a la lista'

In [224]:
data_beer_eroski= pd.DataFrame(index=range(0,len(data_raw_eroski)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_eroski)):
    data_beer_eroski.iloc[beer_number,0] = get_price_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,1] = get_price_liter_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,2] = get_tittle_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,3] = get_promotion_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,4] = get_brand_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,5] = get_container_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,6] = get_volumen_unid_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,7] = get_quantity_pack_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,8] = 'Not image'
    data_beer_eroski.iloc[beer_number,9] = 'Eroski'

In [235]:
data_beer_eroski.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,12.59,1.58,"Cerveza SAN MIGUEL, pack lata 24x33 cl",No promotion,SAN MIGUEL,lata,33 cl,24,Not image,Eroski
1,6.47,1.64,"Cerveza SAN MIGUEL, pack lata 12x33 cl",No promotion,SAN MIGUEL,lata,33 cl,12,Not image,Eroski
2,6.72,1.7,"Cerveza MAHOU Clásica, pack 12x33 cl",No promotion,MAHOU C,No specified,33 cl,12,Not image,Eroski
3,7.75,0.97,"Cerveza MAHOU 5 Estrellas, pack 24x33 cl","13,59 € Antes",MAHOU 5 E,No specified,33 cl,24,Not image,Eroski
4,5.31,1.76,"Cerveza SAN MIGUEL, pack botellín 12x25 cl",2ª unidad -50%,SAN MIGUEL,No specified,25 cl,12,Not image,Eroski
